In [ ]:
import sys
sys.path.append('../src')

import openloop_optimizer as op
import numpy as np
import numpy.polynomial.chebyshev as cheb
import utils 
import datetime

In [ ]:
beta = 0.1

def VDP(t, y, u):
    """Define the Boundary value problem with control parameter u"""
    return np.vstack([
        y[1],
        -y[0] + y[1] * (1 - y[0] ** 2) + u(t),
        - 2 * y[0] + y[3] * (2 * y[0] * y[1] + 1),
        - 2 * y[1] - y[2] - y[3] * (1 -y[0] ** 2)
    ])

def bc(ya, yb):
    """Boundary conditions"""
    return np.array([
        ya[0] - ini[0],
        ya[1] - ini[1],
        yb[2],
        yb[3]
    ])

def gradient(u, p):
    if len(p) != len(u):
        raise ValueError("p and u must have the same length")
    else:
        n = len(p)
        grad = np.zeros(n)
    for i in range(n):
        grad[i] = p[i] + 2 * beta * u[i]
    return grad

def V(grid, u, y1, y2):
    return utils.L2(grid, u) * beta + 0.5 * (utils.L2(grid, y1) + utils.L2(grid, y2))

def gen_bc(ini_val):
    """Generate boundary conditions based on initial values"""
    def bc_func(ya, yb):
        return np.array([
            ya[0] - ini_val[0],
            ya[1] - ini_val[1],
            yb[2],
            yb[3]
        ])
    return bc_func
